In [16]:
import nltk
import string
import import_ipynb
import pandas as pnd
import BggHelper as BH
import BggDbGetter as BDG
import BggDbScraper as BDS
from textblob import TextBlob
from langdetect import detect
from tqdm.notebook import tqdm
from iso_language_codes import *
from nltk.stem import LancasterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from boardgamegeek import BoardGameGeek

In [26]:
class Glossa:
    def __init__(self, dataframe, lang):
        self._dataframe = dataframe
        self._lang = lang
        self._stopwords = set([word.replace("'", "") for word in stopwords.words(language(lang)["Name"].lower())])
        self._punctuation = set(string.punctuation)
    def useless_langs_remover(self, field):
        ## This function removes every row in the dataframe not of the specified language
        for i, row in tqdm((self._dataframe.iterrows())):
            text = row[field]
            if text == "" or text == " " or (not(detect(text) == self._lang)):
                self._dataframe = self._dataframe.drop(index=i)
        self._dataframe.reset_index(drop=True, inplace=True)
        return self
    def stopwords_removal(self, field): 
        for index, row in list(self._dataframe.iterrows()):
            tokenized = word_tokenize(row[field].replace("'", ""))
            self._dataframe[field][index] = ([str(w).lower().replace("'", "") for w in tokenized if not (w in self._stopwords or w in self._punctuation)])
        return self
    def stemming(self, field):
        lancaster = LancasterStemmer()
        for index, row in list(self._dataframe.iterrows()):
            self._dataframe[field][index] = ([str(lancaster.stem(w)) for w in row[field]])
        return self
    def get_df(self):
        return self._dataframe
        

In [27]:
d = {'col1': ["The sky opened up as me and myself shat. it's folly!", "Mi chiamo Mario?", "I don't know where to start!"], 'col2': [0, 1, 2]}
df = pnd.DataFrame(data=d)

In [28]:
a = Glossa(df, "en")

In [29]:
a = (a.useless_langs_remover("col1"))#.stopwords_removal("col1")
a.get_df()

,col1,col2
0,The sky opened up as me and myself shat. it's ...,0
1,I don't know where to start!,2


### Time to preprocess

In [30]:
sw = set(stopwords.words("english"))

In [31]:
language("en")["Name"].lower()

'english'

In [32]:
a.get_df()["col1"]

0    The sky opened up as me and myself shat. it's ...
1                         I don't know where to start!
Name: col1, dtype: object

In [33]:
a = a.stopwords_removal("col1")


/home/tore/venv/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [34]:
a.stemming("col1").get_df()

/home/tore/venv/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,col1,col2
0,"[the, sky, op, shat, fol]",0
1,"[i, know, start]",2
